In [ ]:
%%bash
#pip install TextBlob
#conda install gensim

In [12]:
import pandas as pd
import numpy as np
import textblob as tb
import nltk
from nltk.corpus import stopwords as sw
from nltk.stem import *
import urllib2
from gensim import corpora, models, similarities
import slugify as sl
import pickle
import re

In [3]:
dropbox = "/Users/mr/Dropbox/moviemeta/"
movieplots = pd.read_csv(dropbox + 'movieplots.csv')

In [4]:
def clean(row):
    try:
        row['year'] = int(row['year'])
        return row
    except:
        row['year'] = np.nan
        return row

In [5]:
#this is because CSV conversion has converted list into string
from ast import literal_eval
movieplots['plots'] = movieplots['plots'].apply(literal_eval)

movieplots = movieplots.apply(clean, axis = 1)

In [6]:
dftouse = movieplots[movieplots.year > 2014]

In [7]:
print movieplots.shape
print dftouse.shape

(259028, 5)
(16943, 5)


In [8]:
def slugify (text):
    return sl.slugify(text)

In [9]:
def sentences(text):
    blob = tb.TextBlob(text.decode('unicode-escape', 'ignore'))
    return blob.sentences

In [10]:
def stem(word):
    return PorterStemmer().stem(word)

In [13]:
num = re.compile('\d')
def contains_number(word):
    return bool(num.search(word))

In [14]:
def process(sents, dostop=True, dostem=True, donum=True, doslug=True):
    stopwords = set(sw.words('english'))
    sents_processed  = []
    words_processed  = []
    for sent in sents:
        words = []
        for word in sent.words:
            if donum and contains_number(word):
                    continue
            if doslug:
                word = slugify(word)
            if dostop and word in stopwords:
                    continue
            if dostem:
                word = stem(word)
            if len(word) < 2:
                continue
            words.append(word)
        sents_processed.append(words)
        words_processed += words 
    return (sents_processed, words_processed)

In [15]:
%time
documents = dftouse['plots'].apply(lambda x : process(sentences(' '.join(x)))[1])

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 5.01 µs


In [46]:
#save as pickle
with open(dropbox +'imdb_plots_since_2014.pickle', 'wb') as f:
    np.save(f, documents.values)

In [48]:
#save as text file
with open(dropbox +'imdb_plots_since_2014.list', 'wb') as f:
    for plot in documents:
        f.write("%s\n" % plot)

In [47]:
movies = dftouse['title'].values
with open(dropbox +'imdb_movies_since_2014.pickle', 'wb') as f:
    np.save(f,movies)

In [ ]:
#process with spark
'''import findspark
findspark.init()
import pyspark
conf = (pyspark.SparkConf()
    .setMaster('local')
    .setAppName('pyspark')
    .set("spark.executor.memory", "2g"))
sc = pyspark.SparkContext(conf=conf)
from pyspark.sql import SQLContext
sqlsc=SQLContext(sc)
plots_sdf = sqlsc.createDataFrame(dftouse)
plots = (plots_sdf[['title','plots']]
    .map(lambda x : (' '.join(x[1])))
)
docs = (plots
    .map(sentences)
    .map(process)          
).cache()
docs_sents = docs.map(lambda x : x[0]).collect()
docs_words = docs.map(lambda x : x[1]).collect()
'''